# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine

In [2]:
import re
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

In [4]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jdjarurr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jdjarurr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", con=engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [55]:
def tokenize(text):
    
    # url pattern define
    url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # find and replace urls
    detected_urls = re.findall(url_re, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder") 
    
    #tokenize sentences and lemmatize
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    final_tokens=[]
    for tok in tokens:
        final_tokens = lemmatizer.lemmatize(tok).lower().strip()
    return final_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [56]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [58]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000014FCB590940>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def test_model(y_test, y_pred):
    # this function iterate through the columns and create a classification report
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [16]:
y_pred = pipeline.predict(X_test)

In [18]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.54      0.07      0.12      1501
           1       0.77      0.98      0.87      5003
           2       0.20      0.02      0.04        50

    accuracy                           0.77      6554
   macro avg       0.51      0.36      0.34      6554
weighted avg       0.72      0.77      0.69      6554

request               precision    recall  f1-score   support

           0       0.84      0.98      0.91      5416
           1       0.57      0.13      0.21      1138

    accuracy                           0.83      6554
   macro avg       0.70      0.55      0.56      6554
weighted avg       0.79      0.83      0.78      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

fire               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6489
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

earthquake               precision    recall  f1-score   support

           0       0.92      0.99      0.95      5950
           1       0.69      0.12      0.21       604

    accuracy                           0.91      6554
   macro avg       0.80      0.56      0.58      6554
weighted avg       0.90      0.91      0.89      6554

cold               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6418
           1       0.50      0.01      0.01       136

    accuracy                           0.98      6554
   macro avg       0.74      0.50      0.50      6554
weighted avg       0.97      0.98      0.97      6554

### 6. Improve your model
Use grid search to find better parameters. 

In [45]:
'''Gridsearch takes a lot of time to find optimal combination of hyperparameter.
Thus I am going to try only using a single parameter '''
parameters = {
    'clf__estimator__n_estimators' : [50, 100]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [46]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001975B8B25E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]})

In [47]:
cv.best_params_

{'clf__estimator__n_estimators': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
y_pred = cv.predict(X_test)

In [49]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.44      0.06      0.11      1483
           1       0.77      0.98      0.86      5025
           2       0.17      0.02      0.04        46

    accuracy                           0.76      6554
   macro avg       0.46      0.35      0.34      6554
weighted avg       0.70      0.76      0.69      6554

request               precision    recall  f1-score   support

           0       0.84      0.98      0.90      5426
           1       0.50      0.12      0.19      1128

    accuracy                           0.83      6554
   macro avg       0.67      0.55      0.55      6554
weighted avg       0.78      0.83      0.78      6554

offer               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

storm               precision    recall  f1-score   support

           0       0.92      1.00      0.96      5987
           1       0.61      0.04      0.07       567

    accuracy                           0.91      6554
   macro avg       0.76      0.52      0.51      6554
weighted avg       0.89      0.91      0.88      6554

fire               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6485
           1       0.00      0.00      0.00        69

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

earthquake               precision    recall  f1-score   support

           0       0.92      0.99      0.95      5937
           1       0.69      0.12      0.21       617

    accuracy                           0.91      6554
   macro avg       0.80      0.56      0.58      6554
weighted avg       0.89      0.91      0.88      655

C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [51]:
### Lets Try using AdaBoostClassifier...
pipeline_option2 = Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer()),
                                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
                            ])

In [53]:
pipeline_option2.fit(X_train, y_train)
y_pred_improved = pipeline_option2.predict(X_test)
test_model(y_test, y_pred_improved)

related               precision    recall  f1-score   support

           0       0.31      0.00      0.01      1483
           1       0.77      1.00      0.87      5025
           2       0.20      0.02      0.04        46

    accuracy                           0.77      6554
   macro avg       0.42      0.34      0.30      6554
weighted avg       0.66      0.77      0.67      6554

request               precision    recall  f1-score   support

           0       0.84      0.99      0.91      5426
           1       0.59      0.07      0.13      1128

    accuracy                           0.83      6554
   macro avg       0.71      0.53      0.52      6554
weighted avg       0.79      0.83      0.77      6554

offer               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jdjarurr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

fire               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6485
           1       0.00      0.00      0.00        69

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

earthquake               precision    recall  f1-score   support

           0       0.92      1.00      0.95      5937
           1       0.81      0.12      0.20       617

    accuracy                           0.91      6554
   macro avg       0.86      0.56      0.58      6554
weighted avg       0.91      0.91      0.88      6554

cold               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6424
           1       0.50      0.01      0.02       130

    accuracy                           0.98      6554
   macro avg       0.74      0.50      0.50      6554
weighted avg       0.97      0.98      0.97      6554

### 9. Export your model as a pickle file

In [55]:
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.